In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, ConcatDataset, Dataset
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

data_dir = "/home/23ucs747/mini-project-sem5/dataset_split"
low_conf_csv = "/home/23ucs747/mini-project-sem5/selected_low_confidence.csv"
model_path = "/home/23ucs747/mini-project-sem5/models/efficientnet_b0_base.pth"
save_path = "/home/23ucs747/mini-project-sem5/models/efficientnet_b0_retrained.pth"

Using device: cuda


In [24]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [25]:
train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=train_transforms)
class_names = train_dataset.classes
print(f"Original training data: {len(train_dataset)} images across {len(class_names)} classes.")

Original training data: 3348 images across 4 classes.


In [26]:
class PseudoLabeledDataset(Dataset):
    def __init__(self, csv_file, transform=None, class_names=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.class_to_idx = {cls: i for i, cls in enumerate(class_names)}
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img = Image.open(row["image_path"]).convert("RGB")
        label = self.class_to_idx[row["pred_label"]]  # Use pseudo-label (predicted)
        
        if self.transform:
            img = self.transform(img)
        return img, label

pseudo_dataset = PseudoLabeledDataset(low_conf_csv, transform=train_transforms, class_names=class_names)
print(f"Pseudo-labeled samples loaded: {len(pseudo_dataset)} images.")

Pseudo-labeled samples loaded: 829 images.


In [28]:
combined_dataset = ConcatDataset([pseudo_dataset])
train_loader = DataLoader(combined_dataset, batch_size=16, shuffle=True, num_workers=4)

test_dataset = datasets.ImageFolder(root=f"{data_dir}/test", transform=test_transforms)
full_test=ConcatDataset([train_dataset,test_dataset])
test_loader = DataLoader(full_test, batch_size=16, shuffle=False, num_workers=4)

print(f"Combined training dataset size: {len(combined_dataset)}")


Combined training dataset size: 829


In [29]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 4)
model.load_state_dict(torch.load(model_path))
model.to(device)


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [30]:
for name, param in model.named_parameters():
    param.requires_grad = False  # freeze everything first
for name, param in model.named_parameters():
    if "features.6" in name or "features.7" in name or "classifier" in name:
        param.requires_grad = True  # unfreeze deeper layers

trainable_params = [p for p in model.parameters() if p.requires_grad]
print(f"Unfreezing {len(trainable_params)} parameter groups for retraining.")

Unfreezing 67 parameter groups for retraining.


In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(trainable_params, lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [34]:
num_epochs = 50
best_acc=0.0
best_test=0.0
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 40)

    # Training phase
    model.train()
    train_loss, correct, total = 0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    avg_train_loss = train_loss / len(train_loader)

    # Testing phase
    model.eval()
    test_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_acc = 100 * correct / total
    avg_test_loss = test_loss / len(test_loader)
    if test_acc>best_test:
        best_acc=train_acc
        best_test=test_acc
        torch.save(model.state_dict(), save_path)
        print(f"Retrained model saved at: {save_path}")

    print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Loss:  {avg_test_loss:.4f}, Test Acc:  {test_acc:.2f}%")

    scheduler.step()


Epoch 1/50
----------------------------------------


Retrained model saved at: /home/23ucs747/mini-project-sem5/models/efficientnet_b0_retrained.pth
Train Loss: 0.2812, Train Acc: 89.75%
Test Loss:  0.2108, Test Acc:  93.82%

Epoch 2/50
----------------------------------------


Retrained model saved at: /home/23ucs747/mini-project-sem5/models/efficientnet_b0_retrained.pth
Train Loss: 0.2913, Train Acc: 89.26%
Test Loss:  0.2057, Test Acc:  94.01%

Epoch 3/50
----------------------------------------


Train Loss: 0.2705, Train Acc: 89.38%
Test Loss:  0.2142, Test Acc:  94.01%

Epoch 4/50
----------------------------------------


Train Loss: 0.2727, Train Acc: 89.02%
Test Loss:  0.2117, Test Acc:  93.79%

Epoch 5/50
----------------------------------------


...

Train Loss: 0.2535, Train Acc: 90.59%
Test Loss:  0.2254, Test Acc:  93.72%

Epoch 47/50
----------------------------------------


Train Loss: 0.2428, Train Acc: 92.28%
Test Loss:  0.2165, Test Acc:  93.94%

Epoch 48/50
----------------------------------------


Train Loss: 0.2814, Train Acc: 90.59%
Test Loss:  0.2255, Test Acc:  93.62%

Epoch 49/50
----------------------------------------


Train Loss: 0.2871, Train Acc: 89.63%
Test Loss:  0.2283, Test Acc:  93.60%

Epoch 50/50
----------------------------------------


Train Loss: 0.2841, Train Acc: 87.45%
Test Loss:  0.2263, Test Acc:  93.58%


In [35]:
print(best_acc)
print(test_acc)

90.10856453558505
93.57688634192932
